<a href="https://colab.research.google.com/github/tuonglab/scRNA_WCAI/blob/master/notebook/scRNA_WCAI_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# scRNA-seq ASI2024 pre-conference workshop analysis demo!

Before we dive into the demo, let's first install the necessary packages.

In [ ]:
# setup the notebook
!pip install -qqq scanpy[leiden]
# second thing to do is to clone the repository so that we have all the data and notebooks ready to go
!git clone https://github.com/tuonglab/scRNA_ASI_workshop.git

In [ ]:
# just to make the the code run a bit quieter, i'm suppressing a few warnings here

import warnings

from numba.core.errors import NumbaDeprecationWarning, NumbaWarning

for warn in [
    NumbaDeprecationWarning,
    NumbaWarning,
    FutureWarning,
    UserWarning,
]:  # otherwise very noisy
    warnings.filterwarnings("ignore", category=warn)

# Single-cell RNA seq analysis Demo

This demo will show you the common steps involved to get you started on single cell analysis in Python using [`Scanpy`](https://scanpy.readthedocs.io/en/stable/), the toolkit for analysing single-cell gene expression data.

<a href="https://scanpy.readthedocs.io/en/stable/"><img src="https://scanpy.readthedocs.io/en/stable/_static/Scanpy_Logo_BrightFG.svg" alt="anndata_schema" width="100">


## Preprocessing and Quality Control

First, import packages needed for single-cell RNA seq analysis.

In [ ]:
import os

import scanpy as sc
import pandas as pd

Show current directory:

In [ ]:
os.getcwd()

In [ ]:
# change directory
os.chdir("scRNA_ASI_workshop")
[file for file in os.listdir() if not file.startswith(".")]

Print software versions:

In [ ]:
sc.logging.print_header()

### Reading in files for analysis

For this demo, we have already saved the starting raw datafile as an `.h5ad` file which is a common file format used in single-cell analysis. You can read in the file using the `read_h5ad` function from [`anndata`](https://anndata.readthedocs.io/) package.

This file contains the raw counts of the cells and genes, as well as the metadata associated with the cells and genes.

The file is saved in the `data` folder.


<a href="https://anndata.readthedocs.io/"><img src="https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg" alt="anndata_schema" width="500">

The dataset we will be demo-ing today is on the human prostate.

<a href="https://www.frontiersin.org/journals/endocrinology/articles/10.3389/fendo.2022.1006101/full"><img src="https://www.frontiersin.org/files/Articles/1006101/fendo-13-1006101-HTML-r1/image_m/fendo-13-1006101-g001.jpg" alt="human prostate schema" width="500">


In [ ]:
adata = sc.read_h5ad("data/prostate_demo.h5ad")
adata

In [ ]:
adata.obs

In [ ]:
adata.var

## Standard Quality control

A very common QC step is to assess the mitochondrial content.

High mitochondrial content is often associated with poor quality cells. We can calculate the percentage of mitochondrial genes in each cell and plot it.

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=[
        "mt",
    ],
    inplace=True,
    log1p=True,
)

One can now inspect violin plots of some of the computed QC metrics:

> the number of genes expressed in the count matrix

> the total counts per cell

> the percentage of counts in mitochondrial/ribosomal genes

In [ ]:
sc.pl.violin(
    adata,
    [
        "n_genes_by_counts",
        "total_counts",
        "pct_counts_mt",
    ],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
sc.pl.violin(
    adata,
    [
        "n_genes_by_counts",
        "total_counts",
        "pct_counts_mt",
    ],
    jitter=0.4,
    groupby="name",
    multi_panel=True,
    rotation=90,
)

Additionally, it is useful to consider QC metrics jointly by inspecting a scatter plot colored by `pct_counts_mt`.

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", color="pct_counts_mt")
sc.pl.scatter(adata, x="pct_counts_mt", y="n_genes_by_counts", color="total_counts")

Continue processing with "good" cells only..

In [ ]:
# filter cells if they do not express at least 200 genes
sc.pp.filter_cells(adata, min_genes=200)
# filter genes if they are expressed in at least 3 cells
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
# always check after you have done some filtering to ensure that you are happy with the results
adata

❓Are the cut-offs always appropriate?

## Normalisation

Normalise (library-size correct) the data matrix 𝐗 to 10,000 counts per cell, so that information become comparable between cells.

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

# Logarithmise the data:
sc.pp.log1p(adata)

## Highly Variable Feature/Gene selection

Identify and inspect highly-variable genes

In [ ]:
# (Expects logarithimised data)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

🔔 Count how many is actually highly variable.

In [ ]:
adata.var.highly_variable.value_counts()

In [ ]:
# stash the normalised counts in raw, before we subset to only highly variable genes
adata.raw = adata

## Dimensionality Reduction

### Step 1: Subset to only highly variable genes

In [ ]:
# Actually do the filtering for PCA
adata = adata[
    :, adata.var.highly_variable
].copy()  # all rows/indexes : columns only selecting for those within high_variable, and then making a copy of itself
adata

### Step 2: Regress out effects of "total_counts" per cell and percentage of mitochondrial genes expressed

In [ ]:
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

### Step 3: Scale each gene to unit variance. Clip values exceeding standard deviation of 10.

In [ ]:
sc.pp.scale(adata, max_value=10)

>❓Why do we need to scale the data?

### Step 4: Perform Principal Component Analysis (PCA)

PCA is a linear dimensionality reduction technique that can be used to reduce the dimensionality of the data while retaining most of the variance in the data. It does this by transforming the data into a new set of orthogonal axes called principal components. It is not just used in single-cell analysis but in many other fields as well.

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")

Plot to visualise the variance contribution by each PC


In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

### Step 4: Compute neighbourhood graph

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

### Step 5: Embed the neighbourhood graph using UMAP

UMAP stands for Uniform Manifold Approximation and Projection. It is a non-linear dimensionality reduction technique that is well-suited for preserving local structure in high-dimensional data.

In [ ]:
sc.tl.umap(adata, min_dist=0.3)  # 0.3 is personal preference

#### Visualise UMAP:

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "patient",
        "group",
        "age",
    ],
    ncols=1,
)

### Step 4 and 5 again: Batch correction!

Harmony is a popular batch correction tool that iteratively learns a cell-specific linear correction function at the level of PCA space. It is a very powerful tool for batch correction in single-cell analysis.

It can project ells into a shared embedding in which cells group by cell type rather than dataset-specific conditions, simultaneously accounts for multiple experimental and biological factors.

In [ ]:
# first let's install harmony
!pip install -qqq harmonypy

In [ ]:
sc.external.pp.harmony_integrate(adata, key="barcode")

In [ ]:
adata

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, use_rep="X_pca_harmony")

In [ ]:
sc.tl.umap(adata, min_dist=0.3)  # 0.3 is personal preference

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "patient",
        "group",
        "age",
    ],
    ncols=1,
)

### Step 6: Clustering

We will use the `leiden` algorithm to cluster the cells into different groups. It is a graph-based clustering algorithm that is very popular in single-cell analysis. It is based on optimizing a modularity function that is used to detect communities in networks. It has a resolution parameter that can be tuned to get different levels of granularity in the clustering.

In [ ]:
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
sc.pl.umap(adata, color="leiden")

In [ ]:
sc.pl.umap(adata, color="leiden", legend_loc="on data", legend_fontoutline=2)

### Step 7: Cell type annotation

From here on, the analysis will be more focused on the biological interpretation of the clusters. We will use marker genes to annotate the clusters with cell types.

The way to approach this can vary and I will first show using a statistical test to identify marker genes for each cluster. We will use the `rank_genes_groups` function from `scanpy` to identify marker genes for each cluster.

### 7.1: Rank genes per cluster using Wilcoxon rank-sum test

In [ ]:
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")

Let's visualise the top statistically significant marker genes for each cluster.

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

We can also visualise this as a dot plot.

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=3, color_map="Blues", standard_scale="var", min_logfoldchange=1
)

### 7.2: Manually select marker genes

Ask yourself, what are relevant genes for each cluster? What are the marker genes that you would expect to see?

In [ ]:
marker_genes = {
    "": [
        "",
    ],
}

In [ ]:
sc.pl.dotplot(
    adata, marker_genes, groupby="leiden", standard_scale="var", color_map="Blues"
)

In [ ]:
sc.pl.umap(adata, color=[])

### Transfer the annotations

In [ ]:
cell_types = {
    "0": "",
    "1": "",
    "2": "",
    "3": "",
    "4": "",
    "5": "",
    "6": "",
    "7": "",
    "8": "",
    "9": "",
    "10": "",
    "11": "",
    "12": "",
    "13": "",
    "14": "",
    "15": "",
    "16": "",
    "17": "",
    "18": "",
    "19": "",
}

adata.obs["cell_type"] = [cell_types[x] for x in adata.obs["leiden"]]

In [ ]:
sc.pl.umap(adata, color="cell_type", legend_loc="on data", legend_fontoutline=2)

### 7.3: CellTypist

[`CellTypist`](https://www.celltypist.org/) is a tool that uses a machine learning model to predict cell types based on marker genes. It is a very powerful tool that can be used to predict cell types in single-cell data.

First, install `celltypist` and prepare the data.

In [ ]:
adata.raw.to_adata().write_h5ad("data/for_celltypist.h5ad", compression="gzip")

In [ ]:
!pip install -qqq celltypist
# the session might crash after running this step as we are installing numpy. Don't worry. just reconnect and then continue with the rest below.
!celltypist --update-models

Make a directory for the output after running celltypist on our data

In [ ]:
!mkdir celltypist_output

Run celltypist on our data and allow it to predict labels on each single cell with all the specifications needed.

In [ ]:
!celltypist --indata data/for_celltypist.h5ad --model Immune_All_Low.pkl --outdir celltypist_output --majority-voting

Import and label the object for `predicted_labels.csv` as `celltypist_result`.

In [ ]:
celltypist_result = pd.read_csv("celltypist_output/predicted_labels.csv", index_col=0)
celltypist_result

Transfer the celltypist results to the main data object.

In [ ]:
adata.obs["celltypist_majority_voting"] = pd.Series(
    celltypist_result["majority_voting"]
)
adata.obs

Visualise the data via umap with the new celltypist labels

In [ ]:
sc.pl.umap(adata, color=["celltypist_majority_voting"])
sc.pl.umap(
    adata,
    color=["celltypist_majority_voting"],
    legend_loc="on data",
    legend_fontoutline=2,
)

### Let's save this result

In [ ]:
adata.write_h5ad("data/prostate_demo_processed.h5ad", compression="gzip")

### Let's five a bit deeper into the T cells.

In [ ]:
# adata = sc.read_h5ad("prostate_demo_processed.h5ad")
adata_T = adata[adata.obs["cell_type"] == "T"]
adata_T

Let's re-cluster the T cells and visualise the data. We need to start from the step where most of the gene expression data has been normalised but not filtered yet i.e. is still intact, as we need to calculate the highly variable genes again. This time, the highly variable genes will be specific for the T cell sub-clusters. Lucky for us, we actually did store this step in the `.raw` slot.

### Continue the rest of the steps as before.

In [ ]:
adata_T = adata_T.raw.to_adata()
adata_T

In [ ]:
sc.pp.highly_variable_genes(adata_T, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata_T)

In [ ]:
adata_T.var.highly_variable.value_counts()

In [ ]:
# stash what we have done in raw, before we subset the data
adata_T.raw = adata_T
# Actually do the filtering for PCA
adata_T = adata_T[
    :, adata_T.var.highly_variable
].copy()  # all rows/indexes : columns only selecting for those within high_variable, and then making a copy of itself
adata_T

In [ ]:
sc.pp.regress_out(adata_T, ["total_counts", "pct_counts_mt"])
sc.pp.scale(adata_T, max_value=10)
sc.tl.pca(adata, svd_solver="arpack")
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

In [ ]:
sc.external.pp.harmony_integrate(adata, key="barcode")
sc.pp.neighbors(adata, n_neighbors=10, use_rep="X_pca_harmony")
sc.tl.umap(adata, min_dist=0.3)  # 0.3 is personal preference

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "patient",
        "group",
        "cell_type",
    ],
    ncols=1,
)

In [ ]:
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
sc.pl.umap(adata, color="leiden")
sc.pl.umap(adata, color="leiden", legend_loc="on data", legend_fontoutline=2)

### How would you visualise different T cell marker genes? ❓

Earlier, we saw genes CD4, CD8B. What are other cell surface markers found on T cells?

Some hints from https://www.nature.com/articles/s41467-019-12464-3

In [ ]:
marker_genes = (
    [
        "CD4",
        "CD8B",
        "FOXP3",
        "SELL",
        "CCR7",
        "MKI67",
        "NKG7",
        "GATA3",
        "RORC",
        "CXCR5",
        "CD69",
        "GZMK",
    ],
)
sc.pl.umap(
    adata,
    color=marker_genes,
    ncols=3,
    size=5,
    color_map=tools.cmp(),
)

In [ ]:
sc.pl.dotplot(
    adata_T, marker_genes, groupby="leiden", standard_scale="var", color_map="Blues"
)

In [ ]:
sc.pl.umap(adata_T, color="")

### Transfer the annotations

In [ ]:
cell_types = {
    "0": "",
    "1": "",
    "2": "",
    "3": "",
    "4": "",
    "5": "",
    "6": "",
    "7": "",
    "8": "",
    "9": "",
    "10": "",
    "11": "",
    "12": "",
    "13": "",
    "14": "",
    "15": "",
    "16": "",
    "17": "",
    "18": "",
    "19": "",
}

adata_T.obs["cell_type_T"] = [cell_types[x] for x in adata_T.obs["leiden"]]

In [ ]:
sc.pl.umap(adata_T, color="cell_type_T", legend_loc="on data", legend_fontoutline=2)

In [ ]:
adata_T.write_h5ad("data/prostate_demo_processed_T.h5ad", compression="gzip")

This marks the end of the demo. Good job!

# Other useful resources
For more details on the dataset that we demoed today, please checkout the original publication and data portal:

https://www.prostatecellatlas.org/

<a href="https://doi.org/10.1016/j.celrep.2021.110132"><img src="https://www.prostatecellatlas.org/assets/cover.jpg" alt="prostate_cellrep" width="200">

If you have any questions, email Kelvin at z.tuong@uq.edu.au